## Implementing ML Model Monitoring Pipelines

### Model Performance Drift:
**Description**: Setup a monitoring pipeline to track key performance metrics (e.g., accuracy, precision) of an ML model over time using a monitoring tool or dashboard.

In [1]:
# write your code from here

import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score
from sklearn.datasets import make_classification
from scipy.stats import ks_2samp
from sklearn.model_selection import train_test_split
from scipy.stats import zscore
import unittest

# -------------------------------
# 1. Generate Base & Live Data
# -------------------------------

def generate_dataset(seed=42, drift=False, anomaly=False):
    np.random.seed(seed)
    X, y = make_classification(n_samples=1000, n_features=5, random_state=seed)

    if drift:
        X[:, 0] = X[:, 0] + np.random.normal(2, 0.5, size=X.shape[0])  # Introduce drift in feature 0

    if anomaly:
        y[:20] = 1 - y[:20]  # Flip predictions to simulate bad model outputs

    return pd.DataFrame(X, columns=[f'f{i}' for i in range(X.shape[1])]), pd.Series(y, name='target')

# -------------------------------
# 2. Train a Baseline Model
# -------------------------------

def train_model(X_train, y_train):
    clf = RandomForestClassifier(n_estimators=100, random_state=42)
    clf.fit(X_train, y_train)
    return clf

# -------------------------------
# 3. Monitoring Functions
# -------------------------------

# Performance Drift: Accuracy & Precision monitoring
def monitor_model_performance(y_true, y_pred, baseline_metrics):
    acc = accuracy_score(y_true, y_pred)
    prec = precision_score(y_true, y_pred)
    drift_detected = False

    print(f"[Performance] Accuracy: {acc:.2f}, Precision: {prec:.2f}")
    for metric, current in zip(['accuracy', 'precision'], [acc, prec]):
        baseline = baseline_metrics[metric]
        if abs(current - baseline) > 0.1:  # threshold for drift
            print(f"[ALERT] Drift in {metric}: baseline={baseline:.2f}, current={current:.2f}")
            drift_detected = True
    return drift_detected

# Feature Distribution Drift (KS test)
def monitor_feature_drift(X_train, X_live, threshold=0.05):
    drift_report = {}
    for col in X_train.columns:
        stat, p_value = ks_2samp(X_train[col], X_live[col])
        drift_report[col] = (p_value < threshold)
        if p_value < threshold:
            print(f"[Drift] Feature '{col}' has drifted (p={p_value:.4f})")
    return drift_report

# Anomaly Detection (Z-score of predicted probabilities)
def detect_prediction_anomalies(pred_probs, z_thresh=2.5):
    z_scores = np.abs(zscore(pred_probs))
    anomalies = np.where(z_scores > z_thresh)[0]
    print(f"[Anomaly Detection] {len(anomalies)} anomalies detected")
    return anomalies.tolist()

# -------------------------------
# Run Simulation
# -------------------------------

if __name__ == "__main__":
    # Training phase
    X_train, y_train = generate_dataset()
    X_val, y_val = generate_dataset(seed=100)  # Slightly different data for validation
    model = train_model(X_train, y_train)
    y_val_pred = model.predict(X_val)

    # Save baseline metrics
    baseline_metrics = {
        'accuracy': accuracy_score(y_val, y_val_pred),
        'precision': precision_score(y_val, y_val_pred)
    }

    # Monitoring Phase - Live (with drift and anomalies)
    X_live, y_live = generate_dataset(seed=200, drift=True, anomaly=True)
    y_pred_live = model.predict(X_live)
    pred_probs = model.predict_proba(X_live)[:, 1]

    print("\n=== Monitoring Pipeline ===")
    monitor_model_performance(y_live, y_pred_live, baseline_metrics)
    monitor_feature_drift(X_train, X_live)
    detect_prediction_anomalies(pred_probs)

# -------------------------------
# Unit Tests for Monitoring
# -------------------------------

class TestMonitoringFunctions(unittest.TestCase):

    def test_performance_drift_detection(self):
        base = {'accuracy': 0.9, 'precision': 0.8}
        # Simulate drop in performance
        result = monitor_model_performance([1, 0, 1], [0, 0, 0], base)
        self.assertTrue(result)

    def test_feature_drift(self):
        X1 = pd.DataFrame(np.random.normal(0, 1, (100, 3)), columns=['a', 'b', 'c'])
        X2 = X1.copy()
        X2['a'] += 5  # Add drift
        drift = monitor_feature_drift(X1, X2)
        self.assertTrue(drift['a'])

    def test_anomaly_detection(self):
        probs = np.array([0.1] * 98 + [0.99, 0.001])  # Inject anomalies
        anomalies = detect_prediction_anomalies(probs)
        self.assertGreaterEqual(len(anomalies), 2)

if __name__ == '__main__':
    print("\n=== Running Unit Tests ===")
    unittest.main(argv=[''], exit=False)

F./home/vscode/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
.
FAIL: test_anomaly_detection (__main__.TestMonitoringFunctions)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "/tmp/ipykernel_75325/1438541973.py", line 122, in test_anomaly_detection
    self.assertGreaterEqual(len(anomalies), 2)
AssertionError: 1 not greater than or equal to 2

----------------------------------------------------------------------
Ran 3 tests in 0.008s

FAILED (failures=1)



=== Monitoring Pipeline ===
[Performance] Accuracy: 0.50, Precision: 0.50
[Drift] Feature 'f0' has drifted (p=0.0000)
[Drift] Feature 'f1' has drifted (p=0.0000)
[Drift] Feature 'f2' has drifted (p=0.0000)
[Drift] Feature 'f3' has drifted (p=0.0053)
[Drift] Feature 'f4' has drifted (p=0.0000)
[Anomaly Detection] 1 anomalies detected

=== Running Unit Tests ===
[Anomaly Detection] 1 anomalies detected
[Drift] Feature 'a' has drifted (p=0.0000)
[Performance] Accuracy: 0.33, Precision: 0.00
[ALERT] Drift in accuracy: baseline=0.90, current=0.33
[ALERT] Drift in precision: baseline=0.80, current=0.00


### Feature Distribution Drift:
**Description**: Monitor the distribution of your input features in deployed models to detect any significant shifts from training data distributions.

In [ ]:
# write your code from here

### Anomaly Detection in Predictions:
**DEscription**: Implement an anomaly detection mechanism to flag unusual model
predictions. Simulate anomalies by altering input data.

In [ ]:
# write your code from here